In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from catboost import CatBoostClassifier
import seaborn as sns
import matplotlib.pyplot as plt

# Đọc dữ liệu
df = pd.read_csv(r'D:\MINI_Project\DoAn3\perdict_train_XGBoost\students_grading_dataset_clean.csv')

# Loại bỏ các cột không cần thiết
df.drop(['Student_ID','First_Name','Last_Name','Email','Midterm_Score',
         'Final_Score','Projects_Score','Total_Score'],
        axis=1, inplace=True)

# Tách X, y
X = df.drop('Grade', axis=1)
y = df['Grade']

# Xác định các cột phân loại
cat_features = X.select_dtypes(include='object').columns.tolist()
#print("Categorical features:", cat_features)

# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

"""# Khởi tạo và huấn luyện mô hình
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    verbose=0
)"""
"""model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3.0,
    random_seed=42,
    class_weights=class_weights,
    verbose=100
)"""
model = CatBoostClassifier(verbose=100) #Bắt đầu với giá trị mặc định → test


model.fit(X_train, y_train, cat_features=cat_features)



Learning rate set to 0.084924
0:	learn: 1.3114782	total: 73.6ms	remaining: 1m 13s
100:	learn: 0.7065778	total: 6.06s	remaining: 53.9s
200:	learn: 0.6278538	total: 12.1s	remaining: 48.2s
300:	learn: 0.5602118	total: 18.4s	remaining: 42.6s
400:	learn: 0.5035436	total: 24.9s	remaining: 37.2s
500:	learn: 0.4546906	total: 31.3s	remaining: 31.2s
600:	learn: 0.4158889	total: 37.9s	remaining: 25.2s
700:	learn: 0.3815534	total: 49.4s	remaining: 21.1s
800:	learn: 0.3489834	total: 56.8s	remaining: 14.1s
900:	learn: 0.3214522	total: 1m 3s	remaining: 7.03s
999:	learn: 0.2966206	total: 1m 10s	remaining: 0us


In [7]:
# Dự đoán
y_pred = model.predict(X_test)

# Đánh giá
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Classification Report:
              precision    recall  f1-score   support

           A       0.00      0.00      0.00        32
           B       0.60      0.71      0.65       557
           C       0.51      0.43      0.47       405
           D       0.00      0.00      0.00         6

    accuracy                           0.57      1000
   macro avg       0.28      0.29      0.28      1000
weighted avg       0.54      0.57      0.55      1000



d:\MINI_Project\DoAn3\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\MINI_Project\DoAn3\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\MINI_Project\DoAn3\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier

# 1. Đọc dữ liệu
df = pd.read_csv(r'D:\MINI_Project\DoAn3\perdict_train_XGBoost\students_grading_dataset_clean.csv')

# 2. Loại bỏ các cột không cần thiết
df.drop(['Student_ID','First_Name','Last_Name','Email','Midterm_Score',
         'Final_Score','Projects_Score','Total_Score'],
        axis=1, inplace=True)

# 3. Tách X, y
X = df.drop('Grade', axis=1)
y = df['Grade']

# 4. Xác định cột phân loại (categorical)
cat_features = X.select_dtypes(include='object').columns.tolist()

# 5. Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 6. Tăng trọng số cho các lớp thiểu số (Grade A = 0, D = 3 nếu bạn dùng label encode trước đó)
# ⚠️ Nếu chưa dùng LabelEncoder, bạn có thể dùng map như sau:
label_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
y_train = y_train.map(label_map)
y_test = y_test.map(label_map)

# 7. Thiết lập trọng số (class_weights)
# Giá trị lớn hơn → mô hình học kỹ hơn lớp đó
class_weights = [5.0, 1.0, 1.0, 10.0]  # A và D được ưu tiên

# 8. Huấn luyện mô hình với siêu tham số điều chỉnh

# Khởi tạo và huấn luyện mô hình
# Khởi tạo và huấn luyện mô hình
model = CatBoostClassifier(
    iterations=100,          # Số vòng lặp (số cây sẽ xây dựng). Giá trị càng cao thì mô hình càng học kỹ, nhưng có nguy cơ overfit nếu quá lớn.
    
    learning_rate=0.1,       # Tốc độ học. Giá trị nhỏ giúp mô hình học từ từ, chính xác hơn nhưng cần nhiều iterations hơn. Giá trị lớn học nhanh hơn nhưng có nguy cơ bỏ qua thông tin quan trọng.
    
    depth=6,                 # Độ sâu của mỗi cây quyết định. Độ sâu cao giúp mô hình học được quan hệ phức tạp nhưng dễ bị overfit. Thường chọn từ 4 đến 10.
    
    verbose=0,               # Không hiển thị log trong quá trình huấn luyện (0 = im lặng, 100 = hiển thị mỗi 100 vòng lặp, v.v.). Nếu bạn muốn theo dõi tiến trình, hãy đổi thành `verbose=100`.
    
    class_weights=class_weights  # Danh sách trọng số cho từng lớp. Dùng để xử lý mất cân bằng dữ liệu. Lớp ít mẫu nên có trọng số cao hơn để mô hình ưu tiên học.
)


# 9. Fit model
model.fit(X_train, y_train, cat_features=cat_features)

# 10. Dự đoán
y_pred = model.predict(X_test)

# 11. Báo cáo kết quả
from sklearn.metrics import classification_report

label_names = ['A', 'B', 'C', 'D']
print("\n=== 📊 Classification Report ===")
print(classification_report(y_test, y_pred, target_names=label_names))



=== 📊 Classification Report ===
              precision    recall  f1-score   support

           A       0.00      0.00      0.00        31
           B       0.61      0.76      0.68       562
           C       0.54      0.41      0.46       399
           D       0.00      0.00      0.00         8

    accuracy                           0.59      1000
   macro avg       0.29      0.29      0.29      1000
weighted avg       0.56      0.59      0.57      1000

